In [1]:
import pandas as pd
import numpy as np

# Cyber Data Analytics - Lab 1
Cheatsheet: https://gist.github.com/agalea91/545e2337b94d965be788f7db18b1f497

**Data description**
 - `issuercountrycode`: country where the card was issued
 - `txvariantcode`: the card type that was used (subbrand of visa or master card)
 - `bin`: card issuer identifier
 - `amount`/currencycode: transaction amount in minor units (so 100 EUR = 100 euro cent)
 - `shoppercountrycode`: IP address country
 - `shopperinteraction`: Ecommerce if it was an online transaction, ContAuth if it was a (monthly) subscription
 - `simple_journal`: Payment status. Authorised = “transaction approved and no fraud reported”, Refused = “transaction was declined, can be fraud, but can also be insufficient funds, etc”, Chargeback = “transaction was approved, but turned out to be fraud”
 - `bookingdate`: only relevant for Chargebacks. Time stamp when the chargeback was reported. During simulation you may only use this knowledge after this date. So for example if on an email address a transaction ended in a chargeback, you can only block that email address after the booking date of the chargeback.
 - `cardverificationresponsesupplied`: did the shopper provide his 3 digit CVC/CVV2 code?
 - `cvcresponsecode`: Validation result of the CVC/CVV2 code: 0 = Unknown, 1=Match, 2=No Match, 3-6=Not checked
 - `creationdate`: Date of transaction
 - `accountcode`: merchant’s webshop
 - `mail_id`: Email address
 - `ip_id`: Ip address
 - `card_id`: Card number

**Variables to handle carefully:**

`simple_journal` /
`bookingdate` /
`creationdate`

## 1. Preprocessing

### Load and clean data

In [57]:
# Load data
df = pd.read_csv('data/data_for_student_case.csv', header=0, index_col=0)

# Generate labels
df['labels'] = df['simple_journal'].map({'Chargeback': 1, 'Settled': 0})

# Drop 'Refused' rows
df.drop(df.index[df['simple_journal'] == 'Refused'], inplace=True)

# Drop 'simple_journal' in favor of 'labels'
df.drop(columns='simple_journal', inplace=True)

# Convert datetimes
df['bookingdate'] = pd.to_datetime(df['bookingdate'])
df['creationdate'] = pd.to_datetime(df['creationdate'])

# Drop unneeded columns
# df.drop(columns=["issuercountrycode"], inplace=True)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
df.dtypes

In [55]:
df.sample(10)

,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,labels
txid,,,,,,,,,,,,,,,,
260868,2015-08-04 20:35:42,GB,visadebit,475144.0,7795.0,GBP,GB,Ecommerce,True,1,2015-08-02 20:45:53,UKAccount,email297335,ip45105,card97754,0.0
63392,2015-08-06 13:51:43,AU,mccredit,521729.0,13850.0,AUD,AU,Ecommerce,True,0,2015-08-05 16:43:39,APACAccount,email345852,ip159888,card80735,0.0
400426,2015-10-23 22:54:18,GB,mccredit,522948.0,7895.0,GBP,GB,Ecommerce,True,1,2015-10-23 06:16:29,UKAccount,email166247,ip218170,card130760,0.0
340088,2015-09-18 19:39:47,GB,visadebit,475129.0,7000.0,GBP,GB,Ecommerce,True,1,2015-09-17 18:19:02,UKAccount,email285455,ip98491,card84637,0.0
219271,2015-07-11 00:50:26,GB,visadebit,484431.0,2895.0,GBP,GB,Ecommerce,NaN,1,2015-07-09 22:56:33,UKAccount,email293737,ip182583,card143856,0.0
383899,2015-10-12 01:21:34,GB,visaclassic,450198.0,3185.0,GBP,GB,Ecommerce,True,1,2015-10-09 09:25:15,UKAccount,email110015,ip336047,card260863,0.0
396644,2015-10-20 23:51:15,GB,visadebit,476365.0,6995.0,GBP,GB,Ecommerce,True,1,2015-10-19 17:36:13,UKAccount,email255212,ip342631,card248265,0.0
348969,2015-09-24 02:34:33,GB,mccredit,552213.0,6500.0,GBP,GB,ContAuth,False,0,2015-09-22 19:36:18,UKAccount,email284522,ip279829,card207368,0.0
462660,2015-07-09 20:33:27,SE,mccredit,539881.0,127000.0,SEK,SE,Ecommerce,True,1,2015-07-09 00:17:47,SwedenAccount,email31188,ip231566,card184775,0.0


# 2. Visualization

In [39]:
## 2.1. Class imbalance

In [40]:
## 2.X. Heatmap